In [192]:
import nltk
import pandas as pd
import collections 
import re
import string
import itertools
import sklearn
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = pd.read_csv("Tweets_Mix.csv")

In [3]:
data.head(10)

,Sentiment,Tweets
0,1,you dont have to be crazy to work here serious...
1,0,Paw pawing my ass off smh...im starting to fee...
2,1,and ppl better not act like threatening suicid...
3,0,@juliaroy you are just a tumbling fool. Love it.
4,0,Just gonna go shopping up Fosse Park with mate...
5,1,â youâ€™re stuck with me now iâ€™m not leav...
6,1,i wake up and just want to sleep forever
7,0,@dotmariusz I'm a late bird Mariusz - you fro...
8,1,our friendly purge there is not a better way t...
9,1,this what you a saying many thieves and looter...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
Sentiment    10000 non-null int64
Tweets       10000 non-null object
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [5]:
collections.Counter(data['Sentiment'])

Counter({1: 5000, 0: 5000})

In [6]:
# Writing Function to remove the mentions  URL's  and String with @
def removeURL(text):
    tweet_out = re.sub(r'@[A-Za-z0-9]+','',text)
    re.sub('https?://[A-zA-z0-9]+','',text)
    return tweet_out

# Writing function to remove the non-numeric characters
def removeNonAlphanumeric(text):
    text_out = "".join([char for char in text if char not in string.punctuation])
    return text_out

In [7]:
data["Tweet_No_URL"]  = data["Tweets"].apply(lambda x:removeURL(x))
data["Tweet_No_Punc"] = data["Tweets"].apply(lambda x:removeNonAlphanumeric(x))

In [8]:
data.head()

,Sentiment,Tweets,Tweet_No_URL,Tweet_No_Punc
0,1,you dont have to be crazy to work here serious...,you dont have to be crazy to work here serious...,you dont have to be crazy to work here serious...
1,0,Paw pawing my ass off smh...im starting to fee...,Paw pawing my ass off smh...im starting to fee...,Paw pawing my ass off smhim starting to feel b...
2,1,and ppl better not act like threatening suicid...,and ppl better not act like threatening suicid...,and ppl better not act like threatening suicid...
3,0,@juliaroy you are just a tumbling fool. Love it.,you are just a tumbling fool. Love it.,juliaroy you are just a tumbling fool Love it
4,0,Just gonna go shopping up Fosse Park with mate...,Just gonna go shopping up Fosse Park with mate...,Just gonna go shopping up Fosse Park with mate...


# Tokenization

In [9]:
def tokenization(text):
    token = re.split('\W+',text)
    return token

data ["Tweet_Tokens"] = data["Tweet_No_Punc"].apply(lambda x:tokenization(x))

In [10]:
data

,Sentiment,Tweets,Tweet_No_URL,Tweet_No_Punc,Tweet_Tokens
0,1,you dont have to be crazy to work here serious...,you dont have to be crazy to work here serious...,you dont have to be crazy to work here serious...,"[you, dont, have, to, be, crazy, to, work, her..."
1,0,Paw pawing my ass off smh...im starting to fee...,Paw pawing my ass off smh...im starting to fee...,Paw pawing my ass off smhim starting to feel b...,"[Paw, pawing, my, ass, off, smhim, starting, t..."
2,1,and ppl better not act like threatening suicid...,and ppl better not act like threatening suicid...,and ppl better not act like threatening suicid...,"[and, ppl, better, not, act, like, threatening..."
3,0,@juliaroy you are just a tumbling fool. Love it.,you are just a tumbling fool. Love it.,juliaroy you are just a tumbling fool Love it,"[juliaroy, you, are, just, a, tumbling, fool, ..."
4,0,Just gonna go shopping up Fosse Park with mate...,Just gonna go shopping up Fosse Park with mate...,Just gonna go shopping up Fosse Park with mate...,"[Just, gonna, go, shopping, up, Fosse, Park, w..."
...,...,...,...,...,...
9995,0,@valxx http://twitpic.com/2w1uj - And the artw...,http://twitpic.com/2w1uj - And the artwork's ...,valxx httptwitpiccom2w1uj And the artworks by...,"[valxx, httptwitpiccom2w1uj, And, the, artwork..."
9996,1,i just love reading that im worthless \r\n\r\...,i just love reading that im worthless \r\n\r\...,i just love reading that im worthless \r\n\r\...,"[, i, just, love, reading, that, im, worthless..."
9997,0,Disneyland was great! ... Got to go and pick ...,Disneyland was great! ... Got to go and pick ...,Disneyland was great Got to go and pick up t...,"[, Disneyland, was, great, Got, to, go, and, p..."
9998,1,i hope i die in my sleep,i hope i die in my sleep,i hope i die in my sleep,"[i, hope, i, die, in, my, sleep]"


# Stemming

In [11]:
ps = nltk.PorterStemmer()

def stemming (text):
    out_text = [ps.stem(word) for word in text]
    return out_text

data['FirstDataSetStem'] = data['Tweet_Tokens'].apply(lambda x:stemming(x))

In [12]:
data['FirstDataSetStem'].head()

0    [you, dont, have, to, be, crazi, to, work, her...
1    [paw, paw, my, ass, off, smhim, start, to, fee...
2    [and, ppl, better, not, act, like, threaten, s...
3    [juliaroy, you, are, just, a, tumbl, fool, lov...
4    [just, gonna, go, shop, up, foss, park, with, ...
Name: FirstDataSetStem, dtype: object

# Lemmatizing

In [13]:
wn = nltk.WordNetLemmatizer()

def lemmatize(text):
    out_text = [wn.lemmatize(word) for word in text]
    return out_text

data['FirstDataSet'] =data['Tweet_Tokens'].apply(lambda x:lemmatize(x))

data['FirstDataSet'].head()

0    [you, dont, have, to, be, crazy, to, work, her...
1    [Paw, pawing, my, as, off, smhim, starting, to...
2    [and, ppl, better, not, act, like, threatening...
3    [juliaroy, you, are, just, a, tumbling, fool, ...
4    [Just, gonna, go, shopping, up, Fosse, Park, w...
Name: FirstDataSet, dtype: object

In [14]:
data.head()

,Sentiment,Tweets,Tweet_No_URL,Tweet_No_Punc,Tweet_Tokens,FirstDataSetStem,FirstDataSet
0,1,you dont have to be crazy to work here serious...,you dont have to be crazy to work here serious...,you dont have to be crazy to work here serious...,"[you, dont, have, to, be, crazy, to, work, her...","[you, dont, have, to, be, crazi, to, work, her...","[you, dont, have, to, be, crazy, to, work, her..."
1,0,Paw pawing my ass off smh...im starting to fee...,Paw pawing my ass off smh...im starting to fee...,Paw pawing my ass off smhim starting to feel b...,"[Paw, pawing, my, ass, off, smhim, starting, t...","[paw, paw, my, ass, off, smhim, start, to, fee...","[Paw, pawing, my, as, off, smhim, starting, to..."
2,1,and ppl better not act like threatening suicid...,and ppl better not act like threatening suicid...,and ppl better not act like threatening suicid...,"[and, ppl, better, not, act, like, threatening...","[and, ppl, better, not, act, like, threaten, s...","[and, ppl, better, not, act, like, threatening..."
3,0,@juliaroy you are just a tumbling fool. Love it.,you are just a tumbling fool. Love it.,juliaroy you are just a tumbling fool Love it,"[juliaroy, you, are, just, a, tumbling, fool, ...","[juliaroy, you, are, just, a, tumbl, fool, lov...","[juliaroy, you, are, just, a, tumbling, fool, ..."
4,0,Just gonna go shopping up Fosse Park with mate...,Just gonna go shopping up Fosse Park with mate...,Just gonna go shopping up Fosse Park with mate...,"[Just, gonna, go, shopping, up, Fosse, Park, w...","[just, gonna, go, shop, up, foss, park, with, ...","[Just, gonna, go, shopping, up, Fosse, Park, w..."


# Unique Words

In [15]:
listofwordsLemma = list(itertools.chain.from_iterable(data['FirstDataSet']))
listofwordsStem = list(itertools.chain.from_iterable(data['FirstDataSetStem']))

# Number of Unique Tokens in first Dataset

print('Unique Words after Stemming:'+str(len(set(listofwordsStem))))
print('Unique Words after Lemmatization:'+str(len(set(listofwordsLemma))))

Unique Words after Stemming:17863
Unique Words after Lemmatization:22399


# Stop Words

In [16]:
#nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

In [17]:
def remove_stopWords(token_list):
    text_out = [word for word in token_list if word not in stopwords]
    return text_out

data['SecondDataSet'] = data['FirstDataSetStem'].apply(lambda x:remove_stopWords(x))


In [18]:
data['SecondDataSet']

0       [dont, crazi, work, serious, illeg, requir, do...
1           [paw, paw, ass, smhim, start, feel, better, ]
2       [ppl, better, act, like, threaten, suicid, res...
3                         [juliaroy, tumbl, fool, love, ]
4       [gonna, go, shop, foss, park, mate, got, ï, ½7...
                              ...                        
9995    [valxx, httptwitpiccom2w1uj, artwork, billi, c...
9996                 [, love, read, im, worthless, thank]
9997    [, disneyland, wa, great, got, go, pick, kitti...
9998                                   [hope, die, sleep]
9999                           [dont, want, live, anymor]
Name: SecondDataSet, Length: 10000, dtype: object

In [19]:
# Unique Words After Remove Stop Words 

SecondDataSet = list(itertools.chain.from_iterable(data['SecondDataSet']))

print("Unique Words After Stop Words Removed:"+str(len(set(SecondDataSet))));

Unique Words After Stop Words Removed:17752


# Removing Words with Fewer Frequencies

In [21]:
flat_list = list(itertools.chain.from_iterable(data['SecondDataSet']))

fd = nltk.FreqDist(flat_list)

word_toKeep = list(filter(lambda x:2000>x[1]>3,fd.items()))

word_list_ToKeep = [item[0] for item in word_toKeep]

def remove_lessFreq(tokens):
    text_out = [word for word in tokens if word in word_list_ToKeep]
    return text_out

data['ThirdDataSet'] = data['SecondDataSet'].apply(lambda x:remove_lessFreq(x))

data.head()

,Sentiment,Tweets,Tweet_No_URL,Tweet_No_Punc,Tweet_Tokens,FirstDataSetStem,FirstDataSet,SecondDataSet,ThirdDataSet
0,1,you dont have to be crazy to work here serious...,you dont have to be crazy to work here serious...,you dont have to be crazy to work here serious...,"[you, dont, have, to, be, crazy, to, work, her...","[you, dont, have, to, be, crazi, to, work, her...","[you, dont, have, to, be, crazy, to, work, her...","[dont, crazi, work, serious, illeg, requir, do...","[dont, crazi, work, serious, illeg, requir, do..."
1,0,Paw pawing my ass off smh...im starting to fee...,Paw pawing my ass off smh...im starting to fee...,Paw pawing my ass off smhim starting to feel b...,"[Paw, pawing, my, ass, off, smhim, starting, t...","[paw, paw, my, ass, off, smhim, start, to, fee...","[Paw, pawing, my, as, off, smhim, starting, to...","[paw, paw, ass, smhim, start, feel, better, ]","[ass, start, feel, better]"
2,1,and ppl better not act like threatening suicid...,and ppl better not act like threatening suicid...,and ppl better not act like threatening suicid...,"[and, ppl, better, not, act, like, threatening...","[and, ppl, better, not, act, like, threaten, s...","[and, ppl, better, not, act, like, threatening...","[ppl, better, act, like, threaten, suicid, res...","[ppl, better, act, like, threaten, suicid, res..."
3,0,@juliaroy you are just a tumbling fool. Love it.,you are just a tumbling fool. Love it.,juliaroy you are just a tumbling fool Love it,"[juliaroy, you, are, just, a, tumbling, fool, ...","[juliaroy, you, are, just, a, tumbl, fool, lov...","[juliaroy, you, are, just, a, tumbling, fool, ...","[juliaroy, tumbl, fool, love, ]","[fool, love]"
4,0,Just gonna go shopping up Fosse Park with mate...,Just gonna go shopping up Fosse Park with mate...,Just gonna go shopping up Fosse Park with mate...,"[Just, gonna, go, shopping, up, Fosse, Park, w...","[just, gonna, go, shop, up, foss, park, with, ...","[Just, gonna, go, shopping, up, Fosse, Park, w...","[gonna, go, shop, foss, park, mate, got, ï, ½7...","[gonna, go, shop, park, mate, got, ï]"


# Unique Words After Removing Less Frequency Words

In [22]:
len(set(list(itertools.chain.from_iterable(data['ThirdDataSet']))))

3293

### Feature Extraction

In [23]:
# This vocabluary from data set
UniqueWords = set(list(itertools.chain.from_iterable(data['ThirdDataSet'])))

def join_token(token):
    document = " ".join([word for word in token if not word.isdigit()])
    return document

data['ThirdDataSetDocument'] = data["ThirdDataSet"].apply(lambda x:join_token(x))

#data['ThirdDataSetDocument'].head()
#print(UniqueWords)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

cv =CountVectorizer(UniqueWords)

countvector = cv.fit_transform(data['ThirdDataSetDocument'])

# CountVectorizer Data Frame

countVectorDF = pd.DataFrame(countvector.toarray())
countVectorDF.columns = cv.get_feature_names()

print(countVectorDF)

      1st  2nd  3rd  abil  abl  abort  abov  absenc  absolut  abt  ...  œyou  \
0       0    0    0     0    0      0     0       0        0    0  ...     0   
1       0    0    0     0    0      0     0       0        0    0  ...     0   
2       0    0    0     0    0      0     0       0        0    0  ...     0   
3       0    0    0     0    0      0     0       0        0    0  ...     0   
4       0    0    0     0    0      0     0       0        0    0  ...     0   
...   ...  ...  ...   ...  ...    ...   ...     ...      ...  ...  ...   ...   
9995    0    0    0     0    0      0     0       0        0    0  ...     0   
9996    0    0    0     0    0      0     0       0        0    0  ...     0   
9997    0    0    0     0    0      0     0       0        0    0  ...     0   
9998    0    0    0     0    0      0     0       0        0    0  ...     0   
9999    0    0    0     0    0      0     0       0        0    0  ...     0   

      œyouâ  œã  œðÿ  šã  šðÿ  žã  žðÿ 

# Model Building

# Splitting of Data Set

In [25]:
X = countVectorDF # Features
y= data['Sentiment'] # Target

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=0)

# 1. Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg = logreg.fit(X_train,y_train)

y_pred = logreg.predict(X_test)

In [28]:
logreg.score(X_train,y_train)

0.9895714285714285

In [29]:
logreg.score(X_test,y_test)

0.9623333333333334

# Evaluting the Result using Classification Report

In [30]:
from sklearn import metrics
from sklearn.metrics import classification_report

cnf_mat = metrics.confusion_matrix(y_test,y_pred)
cnf_mat

array([[1453,   38],
       [  75, 1434]], dtype=int64)

In [31]:
print(classification_report(y_test,y_pred))
print("Accurracy:",metrics.accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1491
           1       0.97      0.95      0.96      1509

    accuracy                           0.96      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000

Accurracy: 0.9623333333333334


# Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)

In [33]:
clf.fit(X_train,y_train);

In [34]:
y_pred = clf.predict(X_test)

In [35]:
clf.score(X_train,y_train)

0.9995714285714286

In [36]:
clf.score(X_test,y_test)

0.9583333333333334

In [37]:
cnf_mat = metrics.confusion_matrix(y_test,y_pred)
cnf_mat

array([[1398,   93],
       [  32, 1477]], dtype=int64)

In [38]:
print(classification_report(y_test,y_pred))
print("Accurracy:",metrics.accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      1491
           1       0.94      0.98      0.96      1509

    accuracy                           0.96      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000

Accurracy: 0.9583333333333334


# Support Vector Machine

### TF-IDF 

In [185]:
X = data['Tweet_No_Punc']
y = data['Sentiment']

In [186]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=109)

In [187]:
cv = TfidfVectorizer(min_df=1,stop_words='english')

In [190]:
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [202]:
class_linear = svm.SVC(kernel='linear')
class_linear.fit(X_train_cv,y_train);

In [203]:
y_pred = class_linear.predict(X_test_cv)

In [204]:
class_linear.score(X_train_cv,y_train)

0.9954285714285714

In [205]:
class_linear.score(X_test_cv,y_test)

0.9663333333333334

# Navie Bayes

In [154]:
X = data['Tweet_No_Punc']
y = data['Sentiment']

In [155]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=109)

In [162]:
cv = TfidfVectorizer(min_df=1,stop_words='english')

In [171]:
X_train_cv = cv.transform(X_train)

In [172]:
X_test_cv = cv.transform(X_test)

In [173]:
mnb = MultinomialNB()

In [174]:
mnb.fit(X_train_cv,y_train);

In [175]:
y_pred = mnb.predict(X_test_cv);

In [176]:
mnb.score(X_train_cv,y_train)

0.944

In [177]:
mnb.score(X_test_cv,y_test)

0.9223333333333333

In [178]:
cnf_mat = metrics.confusion_matrix(y_test,y_pred)
cnf_mat

array([[1309,  148],
       [  85, 1458]], dtype=int64)

In [179]:
print(classification_report(y_test,y_pred))
print("Accurracy:",metrics.accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      1457
           1       0.91      0.94      0.93      1543

    accuracy                           0.92      3000
   macro avg       0.92      0.92      0.92      3000
weighted avg       0.92      0.92      0.92      3000

Accurracy: 0.9223333333333333
